In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import Normalizer
from numpy import load

In [4]:
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import IDF
import numpy as np
import pandas as pd

In [5]:
from pyspark.sql.functions import col, collect_list

In [6]:
from pyspark.sql import SparkSession

In [7]:
conf = SparkConf().setMaster("local[*]").setAppName("TFIDF")
sc = SparkContext(conf=conf)

23/02/03 15:31:09 WARN Utils: Your hostname, kittitereh resolves to a loopback address: 127.0.1.1; using 192.168.182.221 instead (on interface wlp2s0)
23/02/03 15:31:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/03 15:31:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
sc

<SparkContext master=local[*] appName=TFIDF>

In [9]:
spark = SparkSession(sc)

In [10]:
spark

In [11]:
train_data = load('trainx16x32_0.npz')['arr_0'][:10000]
test_data = load('testx16x32_0.npz')['arr_0'][:10000]

In [12]:
type(test_data)

numpy.ndarray

In [13]:
test_data.shape

(10000, 2)

In [14]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [15]:
train_df

,0,1
0,0,16981
1,0,23846
2,0,27947
3,0,31189
4,0,31713
...,...,...
9995,15,115548
9996,15,115549
9997,15,115757
9998,15,115826


In [16]:
train_df.dtypes

0    int64
1    int64
dtype: object

In [17]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
mySchema = StructType([ StructField("user", IntegerType(), True),StructField("film", IntegerType(), True)])

In [18]:
train = spark.createDataFrame(train_df, schema=mySchema)
test = spark.createDataFrame(test_df, schema=mySchema)
train.show(5)

/home/kittitereh/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+----+-----+
|user| film|
+----+-----+
|   0|16981|
|   0|23846|
|   0|27947|
|   0|31189|
|   0|31713|
+----+-----+
only showing top 5 rows



In [19]:
train.count()

10000

In [20]:
train.printSchema()

root
 |-- user: integer (nullable = true)
 |-- film: integer (nullable = true)



In [21]:
# find unique users
train.select('user').distinct().rdd.map(lambda r: r[0]).collect()

[1, 2, 0, 3, 5, 4, 6, 8, 7, 9, 10, 11, 12, 13, 15, 14]

In [22]:
films_counted = train.groupBy('film').count()
# find the most watched films
films_counted.orderBy(col("count").desc()).show(5)

+------+-----+
|  film|count|
+------+-----+
|227972|    8|
|189191|    8|
|191991|    7|
|388742|    7|
|236360|    6|
+------+-----+
only showing top 5 rows



In [23]:
films_by_users= train.groupBy("film").agg(collect_list("user"))
films_by_users.show(5)

+----+------------------+
|film|collect_list(user)|
+----+------------------+
|  27|               [4]|
|1169|               [7]|
|1218|               [3]|
|1352|               [9]|
|1937|               [2]|
+----+------------------+
only showing top 5 rows



In [24]:
hashingTF = HashingTF(inputCol="collect_list(user)", outputCol="tf",numFeatures=10000, )
tf = hashingTF.transform(films_by_users)
tf.show()

+----+------------------+--------------------+
|film|collect_list(user)|                  tf|
+----+------------------+--------------------+
|  27|               [4]|(10000,[5102],[1.0])|
|1169|               [7]|(10000,[3707],[1.0])|
|1218|               [3]|(10000,[8051],[1.0])|
|1352|               [9]|(10000,[8889],[1.0])|
|1937|               [2]|(10000,[1574],[1.0])|
|2500|               [2]|(10000,[1574],[1.0])|
|2953|              [12]|(10000,[1024],[1.0])|
|3171|              [12]|(10000,[1024],[1.0])|
|3494|              [13]|(10000,[8648],[1.0])|
|3606|               [3]|(10000,[8051],[1.0])|
|3613|              [15]|(10000,[6477],[1.0])|
|4176|               [7]|(10000,[3707],[1.0])|
|4437|               [2]|(10000,[1574],[1.0])|
|4772|               [5]|(10000,[6466],[1.0])|
|5031|               [9]|(10000,[8889],[1.0])|
|7306|               [5]|(10000,[6466],[1.0])|
|7367|               [7]|(10000,[3707],[1.0])|
|7453|               [7]|(10000,[3707],[1.0])|
|7480|       

In [25]:
idf = IDF(inputCol="tf", outputCol="idf")
tfidf = idf.fit(tf).transform(tf)
tfidf.show(truncate=False)

+----+------------------+--------------------+-----------------------------------+
|film|collect_list(user)|tf                  |idf                                |
+----+------------------+--------------------+-----------------------------------+
|27  |[4]               |(10000,[5102],[1.0])|(10000,[5102],[2.8348152173579018])|
|1169|[7]               |(10000,[3707],[1.0])|(10000,[3707],[2.496547648118015]) |
|1218|[3]               |(10000,[8051],[1.0])|(10000,[8051],[2.2099872807754335])|
|1352|[9]               |(10000,[8889],[1.0])|(10000,[8889],[2.1575162290379803])|
|1937|[2]               |(10000,[1574],[1.0])|(10000,[1574],[2.4286515412855976])|
|2500|[2]               |(10000,[1574],[1.0])|(10000,[1574],[2.4286515412855976])|
|2953|[12]              |(10000,[1024],[1.0])|(10000,[1024],[2.362455874419707]) |
|3171|[12]              |(10000,[1024],[1.0])|(10000,[1024],[2.362455874419707]) |
|3494|[13]              |(10000,[8648],[1.0])|(10000,[8648],[2.3379226878958352])|
|360

In [26]:
# compute L2 norm

from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="idf", outputCol="norm")
data = normalizer.transform(tfidf)

In [27]:
data.show()

+----+------------------+--------------------+--------------------+--------------------+
|film|collect_list(user)|                  tf|                 idf|                norm|
+----+------------------+--------------------+--------------------+--------------------+
|  27|               [4]|(10000,[5102],[1.0])|(10000,[5102],[2....|(10000,[5102],[1.0])|
|1169|               [7]|(10000,[3707],[1.0])|(10000,[3707],[2....|(10000,[3707],[1.0])|
|1218|               [3]|(10000,[8051],[1.0])|(10000,[8051],[2....|(10000,[8051],[1.0])|
|1352|               [9]|(10000,[8889],[1.0])|(10000,[8889],[2....|(10000,[8889],[1.0])|
|1937|               [2]|(10000,[1574],[1.0])|(10000,[1574],[2....|(10000,[1574],[1.0])|
|2500|               [2]|(10000,[1574],[1.0])|(10000,[1574],[2....|(10000,[1574],[1.0])|
|2953|              [12]|(10000,[1024],[1.0])|(10000,[1024],[2....|(10000,[1024],[1.0])|
|3171|              [12]|(10000,[1024],[1.0])|(10000,[1024],[2....|(10000,[1024],[1.0])|
|3494|              [

In [29]:
# multiply the table with itself to get the cosine similarity as the dot product of two by two L2norms
import pyspark.sql.functions as psf
from pyspark.sql.types import DoubleType

dot_udf = psf.udf(lambda x,y: float(x.dot(y)), DoubleType())
res = data.alias("i").join(data.alias("j"), psf.col("i.film") < psf.col("j.film"))\
    .select(
        psf.col("i.film").alias("i"), 
        psf.col("j.film").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .sort("i", "j")


In [30]:
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
mat = IndexedRowMatrix(
    data.select("film", "norm")\
        .rdd.map(lambda row: IndexedRow(row.film, row.norm.toArray()))).toBlockMatrix()
dot = mat.multiply(mat.transpose())

23/02/03 15:49:20 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_91_0 in memory.
23/02/03 15:49:20 WARN MemoryStore: Not enough space to cache rdd_91_0 in memory! (computed 384.0 B so far)
23/02/03 15:49:20 WARN BlockManager: Block rdd_91_0 could not be removed as it was not found on disk or in memory
23/02/03 15:49:20 WARN BlockManager: Putting block rdd_91_0 failed
23/02/03 15:49:35 ERROR Executor: Exception in task 0.0 in stage 34.0 (TID 85)
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix.$anonfun$toBlockMatrix$7(IndexedRowMatrix.scala:159)
	at org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix$$Lambda$3833/0x000000084136e040.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.storage.memory.PartiallyUnrolledIterator.next(MemoryStore.scala:783)
	at 

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 57612)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/kittitereh/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/home/kittitereh/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/home/kittitereh/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 257, in accum_updates
    num_

ConnectionRefusedError: [Errno 111] Connection refused

In [184]:
res = tfidf.rdd.map(lambda x : (x.film, x["collect_list(user)"], x.tf, x.idf,(None if x.idf is None else x.idf.values.sum())))

In [185]:
for r in res.take(10):
    print(r)

(27, [4], SparseVector(10000, {5102: 1.0}), SparseVector(10000, {5102: 2.8348}), 2.8348152173579018)
(1169, [7], SparseVector(10000, {3707: 1.0}), SparseVector(10000, {3707: 2.4965}), 2.496547648118015)
(1218, [3], SparseVector(10000, {8051: 1.0}), SparseVector(10000, {8051: 2.21}), 2.2099872807754335)
(1352, [9], SparseVector(10000, {8889: 1.0}), SparseVector(10000, {8889: 2.1575}), 2.1575162290379803)
(1937, [2], SparseVector(10000, {1574: 1.0}), SparseVector(10000, {1574: 2.4287}), 2.4286515412855976)
(2500, [2], SparseVector(10000, {1574: 1.0}), SparseVector(10000, {1574: 2.4287}), 2.4286515412855976)
(2953, [12], SparseVector(10000, {1024: 1.0}), SparseVector(10000, {1024: 2.3625}), 2.362455874419707)
(3171, [12], SparseVector(10000, {1024: 1.0}), SparseVector(10000, {1024: 2.3625}), 2.362455874419707)
(3494, [13], SparseVector(10000, {8648: 1.0}), SparseVector(10000, {8648: 2.3379}), 2.3379226878958352)
(3606, [3], SparseVector(10000, {8051: 1.0}), SparseVector(10000, {8051: 2.21

In [186]:
from pyspark.sql.functions import udf

sum_ = udf(lambda v: float(v.values.sum()), DoubleType())
tfidf.withColumn("idf_sum", sum_("idf")).show()

+----+------------------+--------------------+--------------------+------------------+
|film|collect_list(user)|                  tf|                 idf|           idf_sum|
+----+------------------+--------------------+--------------------+------------------+
|  27|               [4]|(10000,[5102],[1.0])|(10000,[5102],[2....|2.8348152173579018|
|1169|               [7]|(10000,[3707],[1.0])|(10000,[3707],[2....| 2.496547648118015|
|1218|               [3]|(10000,[8051],[1.0])|(10000,[8051],[2....|2.2099872807754335|
|1352|               [9]|(10000,[8889],[1.0])|(10000,[8889],[2....|2.1575162290379803|
|1937|               [2]|(10000,[1574],[1.0])|(10000,[1574],[2....|2.4286515412855976|
|2500|               [2]|(10000,[1574],[1.0])|(10000,[1574],[2....|2.4286515412855976|
|2953|              [12]|(10000,[1024],[1.0])|(10000,[1024],[2....| 2.362455874419707|
|3171|              [12]|(10000,[1024],[1.0])|(10000,[1024],[2....| 2.362455874419707|
|3494|              [13]|(10000,[8648],[1.0